In [ ]:
import pandas as pd
import numpy as np
import linopy

In [ ]:
snapshots = pd.Index(range(4))
snapshots.name = "snapshot"

In [ ]:
d = pd.Series([2,4,5,4], snapshots, name="demand")

In [ ]:
e_nom = 5

In [ ]:
e_in = pd.Series([4, 5, 4, 2], snapshots)

In [ ]:
def build_full_model(
    snapshots,
    d,
    e_in,
    e_nom
):

    m = linopy.Model()

    e = m.add_variables(lower=0, coords=[snapshots], name="e")

    p = m.add_variables(lower=0, coords=[snapshots], name="p")

    ls = m.add_variables(lower=0, coords=[snapshots], name="ls")

    m.add_constraints(e <= e_nom, name="storage_capacity")

    m.add_constraints(e - e.shift(snapshot = 1) - e.shift(snapshot = -3) + p == e_in, name="storage_balance")

    m.add_constraints(p + ls == d, name="power_balance")

    m.add_objective(ls.sum()*100)
    
    return m

In [ ]:
f = build_full_model(
    snapshots, 
    d, 
    e_in,
    e_nom
)

In [ ]:
f.solve(solver_name="cbc")

In [ ]:
e_initial = pd.Series([0,0,0,0], snapshots)

In [ ]:
e_final = pd.Series([0, 0, 0, 0], snapshots)

In [ ]:
def build_lower_model(
    snapshots,
    d,
    e_in,
    e_initial,
    e_final,
    e_nom
):

    m = linopy.Model()

    e = m.add_variables(lower=0, coords=[snapshots], name="e")

    p = m.add_variables(lower=0, coords=[snapshots], name="p")

    ls = m.add_variables(lower=0, coords=[snapshots], name="ls")
    
    su = m.add_variables(lower=0, coords=[snapshots], name="su")
    sd = m.add_variables(lower=0, coords=[snapshots], name="sd")

    m.add_constraints(e <= e_nom, name="storage_capacity")

    m.add_constraints(e - e.shift(snapshot = 1) + p + su - sd == e_in + e_initial - e_final, name="storage_balance")
    
    #m.add_constraints(e >= e_final - e_in, name="end_storage_level")

    m.add_constraints(p + ls == d, name="power_balance")

    m.add_objective(ls.sum()*100 + su.sum()*1e3 + sd.sum()*1e3)
    
    return m

In [401]:
l = build_lower_model(
    snapshots,
    d,
    e_in,
    e_initial,
    e_final,
    e_nom
)

In [402]:
l.solve(solver_name="cbc")

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc -printingOptions all -import /tmp/linopy-problem-dz9i2h__.lp -solve -solu /tmp/linopy-solve-qkjzumyy.sol (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 4 (-8) rows, 15 (-5) columns and 18 (-13) elements
Perturbing problem by 0.001% of 1000 - largest nonzero change 0.00010234913 ( 1.7802805e-05%) - largest zero change 9.3832383e-05
0  Obj 0 Primal inf 5.999996 (4)
3  Obj 0.00063483868
Optimal - objective value 0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 3 iterations time 0.022, Presolve 0.01
Total time (CPU seconds):       0.14   (Wallclock seconds):       0.02



('ok', 'optimal')

In [479]:
up = linopy.Model()

snapshots_up = pd.Index([1,3])

snapshots_up.name = "snapshot"

e = up.add_variables(lower=0, coords= [snapshots_up], name="e")

theta = up.add_variables(lower=-1e7, name="theta")

up.add_constraints(e <=  e_nom , name="capacity_constraint")

up.add_objective(theta*1)

LinearExpression
----------------
+1 theta

In [507]:
p = {}

for i in range(2):
    p[i] = build_lower_model(
        snapshots[i*2:i*2+2],
        d.loc[snapshots[i*2:i*2+2]],
        e_in.loc[snapshots[i*2:i*2+2]],
        e_initial.loc[snapshots[i*2:i*2+2]],
        e_final.loc[snapshots[i*2:i*2+2]],
        e_nom   
    )

    p[i].solve(solver_name="cbc")

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc -printingOptions all -import /tmp/linopy-problem-e7_v96dj.lp -solve -solu /tmp/linopy-solve-18d3tw3n.sol (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2 (-4) rows, 7 (-3) columns and 8 (-7) elements
0  Obj 0 Primal inf 3.999998 (2)
1  Obj 2e-18
Optimal - objective value 0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 1 iterations time 0.022, Presolve 0.01
Total time (CPU seconds):       0.05   (Wallclock seconds):       0.00

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc -printingOptions all -import /tmp/linopy-problem-0pnuq9pi.lp -solve -solu /tmp/linopy-solve-6pxqp38w.sol (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2 (-4) rows, 7 (-3) columns and 8 (-7) elements
0  Obj 0 Primal inf 3.999998 (2)
1  Obj 2e-18
Optimal - 

In [508]:
cut = {}

for i in range(2):
    
    cut_pars = {}
    
    h = {}
    u = {}
    T = {}

    variables = [0,1]

    for c in p[i].constraints:
        h[c] = p[i].constraints[c].rhs.to_dataframe()
        u[c] = p[i].constraints[c].dual.to_dataframe()
        
        #h[c].loc[h[c].index, "rhs"] = e_in.loc[h[c].index]

        T_constraint= pd.DataFrame()
        for var in variables:

            T_constraint[var] = (
                (
                    p[i].constraints[c].vars.to_dataframe()
                    .unstack("_term") 
                    == var)
                .multiply(1)["vars"]
                .multiply(
                    p[i].constraints[c].coeffs.to_dataframe()
                    .unstack("_term")
                    ["coeffs"]
                )
            ).sum(axis=1)

        T_constraint.columns = T_constraint.columns + 2*i

        T_constraint.columns = "e" + T_constraint.columns.astype(str)

        T[c] = T_constraint

    T["storage_balance"]["e" + str(d.iloc[[i*2-1]].index[0])] = [-1, 0]
    
    cut_pars["h"] = h
    cut_pars["T"] = T
    cut_pars["u"] = u

    cut[i] =cut_pars


In [509]:
T_all = {}
h_all = {}
u_all = {}

for c in p[i].constraints:

    T_all[c] = pd.concat([cut[i]["T"][c] for i in range(2)]).fillna(0).sort_index(axis=1)
    u_all[c] = pd.concat([cut[i]["u"][c] for i in range(2)]).fillna(0).sort_index(axis=1)
    h_all[c] = pd.concat([cut[i]["h"][c] for i in range(2)]).fillna(0).sort_index(axis=1)

h_all["storage_balance"].rhs += e_final - e_initial

In [510]:
e_cut = sum(h_all[c].rhs.multiply(u_all[c].dual).sum() for c in p[i].constraints)

In [511]:
var_u = ["e1", "e3"]

In [512]:
E_cut = sum(T_all[c][var_u].multiply(u_all[c].dual,axis=0).sum() for c in p[i].constraints)

In [513]:
E_cut.index=[1,3]

In [514]:
E_cut.index.name = "snapshot"

In [515]:
up.add_constraints((E_cut.to_xarray()*e).sum() + theta >= e_cut)

Constraint `con2`
-----------------
+0 e[1] + 0 e[3] + 1 theta ≥ -0.0

In [516]:
up.solve(solver_name="cbc")

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc -printingOptions all -import /tmp/linopy-problem-man_a43r.lp -solve -solu /tmp/linopy-solve-ebexy9mo.sol (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2 (-3) rows, 3 (0) columns and 6 (-3) elements
0  Obj 0 Primal inf 2.999999 (1)
1  Obj 3e-18
Optimal - objective value 0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 1 iterations time 0.012, Presolve 0.00
Total time (CPU seconds):       0.03   (Wallclock seconds):       0.00



('ok', 'optimal')

In [517]:
e_final = up.solution.e.to_dataframe().reindex(e_final.index, fill_value=0).e

In [518]:
e_initial = (
    up.solution.e.to_dataframe()
    .reindex(e_initial.index, fill_value=0)
    .shift(1, fill_value=0) 
    + up.solution.e.to_dataframe()
    .reindex(e_initial.index, fill_value=0)
    .shift(-3, fill_value=0)
).e

In [519]:
up.objective_value

0.0